In [ ]:
from diffmain import *
from DataL import *
from torchsummary import summary
from simpleAE import myAE
import glob
import sys
import os
import re
import torch    
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pdb
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
from vae import *
import DataL
from sketch_condition_unet import *

In [ ]:
transform2 = transforms.Compose([transforms.ToTensor()]) 
SketchySet = get_sketchimageclass('crocodilian', transform=transform2, size=64) #SketchyData(sketches_dir, photos_dir, transform=transform)

#torch.utils.data.DataLoader(SketchySet, batch_size=256)
print(len(SketchySet))

i=random.randint(0,len(SketchySet)-1)
sk,im = SketchySet[i]

# Convert the tensors back to numpy arrays
sk = sk.numpy()
im = im.numpy()

sk=np.array(sk).transpose((1,2,0))
im=np.array(im).transpose((1,2,0))
#print(im)
fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].imshow(sk)
ax[0].set_title('Sketch')
ax[0].axis('off')

ax[1].imshow(im)
ax[1].set_title('image')
ax[1].axis('off')

plt.show()

In [ ]:
diffusion_steps = 500
sampling_steps = 250
DDIM = True


seed = 112233
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
unet = smallSketchUnet(
    image_size=64,
    in_channels=6,
    out_channels=3).to(device)

#model.load_state_dict(torch.load('model.pth', map_location=torch.device('cpu')))   

diffusion = GuassianDiffusion(diffusion_steps, device = device)

lr = 0.005
ngpus = torch.cuda.device_count()
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12355"
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
local_rank = int(os.getenv("LOCAL_RANK", "0"))

device = "cuda:{}".format(local_rank)
torch.cuda.set_device(device)
torch.manual_seed(seed + local_rank)
np.random.seed(seed + local_rank)
batch_size = 80

if torch.distributed.is_initialized():
    torch.distributed.destroy_process_group()

if ngpus > 1:
    if local_rank == 0:
        print(f"Using distributed training on {ngpus} gpus.")
    batch_size = batch_size // ngpus
    torch.distributed.init_process_group(backend="nccl", init_method="env://")
    unet = DDP(unet, device_ids=[local_rank], output_device=local_rank)

#train_set=create_smaller_dataset(get_miniset(), factor=50)

sampler = DistributedSampler(SketchySet) if ngpus > 1 else None

train_loader = DataLoader(SketchySet, batch_size=batch_size, sampler=sampler, num_workers=1, pin_memory=True)

if local_rank == 0:
    print(
        f"Training dataset loaded: Number of batches: {len(train_loader)}, Number of images: {len(SketchySet)}"
    )

In [ ]:
ae = myAE(conv_out_size = 3, latent_dim = 100, variation = False).to(device)

class big_model(nn.Module):
    def __init__(self, ae, unet):
        super(big_model, self).__init__()
        self.ae = ae
        self.unet = smallSketchUnet
        
    def forward(self, sketch, condition):
        encoded = self.encode(sketch)
        #print("Encoded setch shape", encoded.shape)
        t = torch.randint(diffusion.timesteps, (len(encoded),), dtype=torch.int64).to(device)
        #print("T shaq: ", t.shape)
        xt, eps = diffusion.sample_from_forward_process(encoded, t)
        #print("XT shep", xt.shape)
        assert(xt.shape == condition.shape)
        pred_eps = self.unet(xt, t, sketch=condition)
        pred_latent_im = diffusion.sample_from_reverse_process(self.unet, xt, diffusion.timesteps, ddim=True)
        decoded = self.decode(pred_latent_im)
        
        return eps, pred_eps, decoded

    def encode(self,x):
        return self.ae.encoder(x)

    def decode(self,x):
        return self.ae.decoder(x)

def lossf(e, prede, im, predim):
    assert(e.shape == prede.shape)
    assert(im.shape == predim.shape)
    loss=  ((prede - e) ** 2).mean() + ((im - predim) **2 ).mean()
    return loss